In [1]:
%reload_ext autoreload
%autoreload 2

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from collections import OrderedDict
import numpy as np
import re
import h5py
from functools import partial
from itertools import product

from calibration import Display


# Wokflow

* General set up (Tab 0):
   
   - Module Number: Specify channel/module numbers that you want to analyse. Must be comma separated.
   - Pulse indices: Pulses that have x-rays. You can provide a range (start:stop:step) or comma separated (1,3,4 etc) or combination of range and individual pulses (1:10:2, 11, 14, 16 etc)
   - Region of interest (ROI x) in x for each module: Only range based (start:stop)
   - Region of interest (ROI y) in y for each module: Only range based (start:stop)
   
* Dark Run (Tab 1):
    
    - Dark Run Folder: (str) /gpfs/exfel/exp/MID/201931/proposal/raw/run_number
    - Train ids: Train indices to take average over. Range based (start:stop). Deafult is all train (:)
    - Process Darak button: To evalute average. This is done in parallel over modules. Doesn't block
      further analysis. Once results (average image and histograms) are available it be displayed automatically.
    - Using Pulses (slider) and module numbers (dropbox) on top one can visulaize data for each pulses or modules.

* Data Visualization (Tab 2):
    
    - Run Folder: (str) /gpfs/exfel/exp/MID/201931/proposal/raw/run_number
    - Train ids: Train indices. Range based (start:stop). Use cautiosly. It loads all train data in memory.
    Also processing is parallized over modules, therefore maximum pickle size has to be respected.
    
    - Subtract Dark. Once dark average data is available you will be able to check on subtract dark.
    - Load Run: Once things are set up, load the run.
    
    - Fitting Procedure:
    
        - First chose some reasonable peak threshold to filter out number of peaks
        - Chose peak distance to remove very close peaks,
        - Once peaks are chosen, one can click on button Fit Histogram to fit Gaussians. Number of peaks define the number of gaussian functions that will be used to fit the histogram. Generally fitting with 3-4 peaks give reasonable fitting 

In [2]:
config = dict(
    dark_run_folder='/gpfs/exfel/exp/MID/201931/p900091/raw/r0504',
    run_folder='/gpfs/exfel/exp/MID/201931/p900091/raw/r0491',
)

## Widgets Display

In [3]:
d = Display(config=config)
d.control_panel()

Output(layout=Layout(border='1px solid', height='160px', width='100%'))

# Analysis without GUI



In [4]:
from functools import partial
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from scipy.ndimage import gaussian_filter, gaussian_filter1d
from scipy.signal import find_peaks

from calibration import DataProcessing, gauss_fit, eval_statistics
from karabo_data import by_index

    
    def DataProcessing(module_number, path, *,
                       train_index=None, pulse_ids=None,
                       rois=None, operation=None,
                       dark_run=None):
                   
           """ Process Data

                    Parameters
                    ----------
                    module_number: int
                        Channel number between 0, 15
                    path: str
                        Path to Run folder
                    train_index: karabo_data (by_index)
                        Default (all trains by_index[:])
                    pulse_ids: str
                        For eg. ":" to select all pulses in a train
                                "start:stop:step" to select indices with certain step size
                                "1,2,3" comma separated pulse index to select specific pulses
                                "1,2,3, 5:10" mix of above two
                        Default: all pulses ":"
                    rois: karabo_data slice constructor by_index
                        Select ROI of image data. For eg. by_index[..., 0:128, 0:64]
                        See karabo_data method: `get_array`

                    operation: function
                        For eg. functools.partial(np.mean, axis=0) to take mean over trains
                    dark_run: ndarray
                        dark_data to subtract

                    Return
                    ------
                    out: ndarray
                        Shape:  operation -> (n_trains, n_pulses, ..., slow_scan, fast_scan)
                    """

# General Parameters

In [5]:
module_number = 15
pulse_ids = "1:250:2"
rois = by_index[:, :]

# Dark Run average. Do not run if dark data is already evaluated and stored in file. Jump to visualization cell

In [6]:
# parameters

dark_run_folder = "/gpfs/exfel/exp/MID/201931/p900091/raw/r0504"

dark_train_index = by_index[:]

# mean over train index 
operation = partial(np.mean, axis=0)


In [7]:
dark_average = DataProcessing(
    module_number, 
    dark_run_folder,
    train_index=dark_train_index,
    pulse_ids=pulse_ids,
    rois=rois,
    operation=operation)

print(dark_average.shape)

(125, 2, 512, 128)


# Write dark data to file

In [8]:
# Write dark data to file that can be used later

dark_run = {module_number:dark_average}

with h5py.File("dark_run.h5", "w") as f:
    for modno, data in dark_run.items():
        g = f.create_group(f"entry_1/instrument/module_{modno}")
        g.create_dataset('data', data=dark_run[modno])

# Subtract Dark from a run. Do not run if dark subtracted data is already evaluated and stored in file. Jump to visualization cell

In [12]:

run_folder = "/gpfs/exfel/exp/MID/201931/p900091/raw/r0491"

proc_train_index = by_index[250:450]



In [13]:
dark_subtracted = DataProcessing(
    module_number, 
    run_folder,
    train_index=proc_train_index,
    pulse_ids=pulse_ids,
    rois=rois,
    dark_run=dark_average)

In [15]:
# n_trains, pulses, gain_bits, slow_scan, fast_scan
print(f"Shape of Dark Subtracted Data: {dark_subtracted.shape}")

Shape of Dark Subtracted Data: (200, 125, 2, 512, 128)


# Write dark subtracted Data to file

In [16]:
dark_sub_run = {module_number:dark_subtracted}

with h5py.File("/gpfs/exfel/data/scratch/kamile/dark_subtracted.h5", "w") as f:
    for modno, data in dark_sub_run.items():
        g = f.create_group(f"entry_1/instrument/module_{modno}")
        g.create_dataset('data', data=dark_sub_run[modno])

# Visualization 

In [41]:
# Create Empty Plots

import ipywidgets as widget

image_widget = go.FigureWidget(data=go.Heatmap(showscale=False))

trace = [go.Bar(name="Data"),
         go.Scatter(mode='markers', name="Peaks", marker=dict(size=10)), 
         go.Scatter(mode='lines', name="Scipy Fit"),
         go.Scatter(mode='lines+markers', name="Minuit Fit")]

hist_widget = go.FigureWidget(data=trace)

residual_widget = go.FigureWidget(data=[
    go.Box(boxmean='sd', name='Scipy', boxpoints=False),
    go.Box(boxmean='sd', name='Minuit', boxpoints=False),])

fit_params_widget = go.FigureWidget(
    data=go.Table(
        header=dict(
            values=["Positions", "Amplitudes", "Width"],
            fill_color='paleturquoise',
            align='center'),
        cells=dict(
            fill_color='lavender',
            align='left')))


image_widget.layout.update(margin=dict(l=0, b=40, t=50), width=450)
hist_widget.layout.update(margin=dict(r=0, l=10, b=40, t=50), width=450)
residual_widget.layout.update(margin=dict(l=0, b=40, t=50), width=450)
fit_params_widget.layout.update(margin=dict(l=0, b=40, t=50), 
                                width=900, height=300, title="Scipy Fit Info.")


Layout({
    'height': 300,
    'margin': {'b': 40, 'l': 0, 't': 50},
    'template': '...',
    'title': {'text': 'Scipy Fit Info.'},
    'width': 900
})

# Read dark subtracted data from file

In [18]:
with h5py.File("/gpfs/exfel/data/scratch/kamile/dark_subtracted.h5", "r") as f:
    dark_subtracted = np.moveaxis(f["entry_1/instrument/module_15/data"][:], 0, 1)

print(f"Shape of Dark subtracted data {dark_subtracted.shape}")

Shape of Dark subtracted data (125, 200, 2, 512, 128)


In [43]:
# Clear plots
hist_widget.data[1].x = []
hist_widget.data[1].y = []
hist_widget.data[2].x = []
hist_widget.data[2].y = []
hist_widget.data[3].x = []
hist_widget.data[3].y = []

# Pulse index to visualize
pulse_id = 0
bins=600


# Mean image over trains (In this case 30 trains)
image_widget.data[0].z = np.mean(dark_subtracted[pulse_id, :, 0, ...], axis=0)

# Evaluate histogram
bin_centers, bin_counts = eval_statistics(dark_subtracted[:, pulse_id, 0, ...], bins=bins)

# Update histogram image

hist_widget.data[0].x = bin_centers
hist_widget.data[0].y = bin_counts

display(widget.HBox([image_widget, hist_widget]))

    'data': [{'showscale': False,
              'type': 'heatmap',
             …

# Fitting dark subtracted data

 

###   Peak finding to estimate intial fit parameters
    
    * Apply Gaussian filter to smooth out the histogram
    * Evaluate peaks for this filtered data
        - Peaks parameters: height, distance

In [46]:
# Clear plots
hist_widget.data[2].x = []
hist_widget.data[2].y = []
hist_widget.data[3].x = []
hist_widget.data[3].y = []

# Peak params
peak_threshold = 200
peak_distance = 20

# Apply Gaussian filter
filtered = gaussian_filter(bin_counts, 1.5)
#filtered = bin_counts

# Evaluate peaks
peaks, _ = find_peaks(filtered,
                      height=peak_threshold,
                      distance=peak_distance)

#Plot peaks and Gaussian filtered curve. 

hist_widget.data[1].x = bin_centers[peaks]
hist_widget.data[1].y = filtered[peaks]

print(f"Number of peaks: {len(peaks)}")
print(f"Peak positions, amplitudes: {list(zip(bin_centers[peaks], filtered[peaks]))}")

display(widget.HBox([image_widget, hist_widget]))

Number of peaks: 3
Peak positions, amplitudes: [(-25.435997, 77840), (14.431009, 97217), (57.364708, 63586)]


    'data': [{'showscale': False,
              'type': 'heatmap',
             …

# Benchmarking IMINUIT and SCIPY


    

In [47]:
from iminuit import Minuit
from iminuit import minimize
from scipy.optimize import curve_fit


In [48]:
def gaussian(x, *params):
    num_gaussians = int(len(params) / 3)
    A = params[:num_gaussians]
    w = params[num_gaussians:2*num_gaussians]
    c = params[2*num_gaussians:3*num_gaussians]
    y = sum(
        [A[i]*np.exp(-(x-c[i])**2./(w[i])) for i in range(num_gaussians)])
    
    return y

def least_squares_np(xdata, ydata,  params):  
    var = 1.0
    y = gaussian(xdata, *params) 
    return np.sum((ydata - y) ** 2) / xdata.shape[0]

In [49]:
# Function to pass to Minuit
least_sq = partial(
    least_squares_np, bin_centers, filtered)


# Construct initial fit params list
params = []

params.extend(filtered[peaks])                              # Extend Amplitudes [A1, A2, A3, ...]
params.extend(np.full((len(bin_centers[peaks])), 100))      # Extend Sigma [S1, S2, S3, ...]
params.extend(bin_centers[peaks])                           # Extend Positions [P1, P2, P3, ...]


# Construct bounds for parameters MINUIT format
bounds_minuit = [(0, None) if i < 2 * (len(params) // 3) else (None, None) 
                  for i in range(len(params))]

# Construct bounds for parameters SCIPY format
bounds_scipy = [[ 0 if  i < 2 * (len(params) // 3) else -np.inf 
                 for i in range(len(params))], 
                np.inf]

popt = params
perr = None
# Scipy Fitting
try:
    popt, pcov = curve_fit(
        gaussian, bin_centers, filtered, p0=params, bounds=bounds_scipy)
    perr = np.sqrt(np.diag(pcov))
except Exception as ex:
    pass

# Minuit Fitting
m = Minuit.from_array_func(
    least_sq, params, error=0.1, errordef=1, limit=tuple(bounds_minuit))

minuit_res = m.migrad()

# Evaluation of residuals
residual_scipy = filtered - gaussian(bin_centers, *popt)
residual_minuit = filtered - gaussian(bin_centers, *m.np_values())


# Update Plots
hist_widget.data[2].x = bin_centers
hist_widget.data[2].y = gaussian(bin_centers, *popt)

hist_widget.data[3].x = bin_centers
hist_widget.data[3].y = gaussian(bin_centers, *m.np_values())

residual_widget.data[0].y = residual_scipy
residual_widget.data[1].y = residual_minuit

if perr is not None:
    table_elements = [f"{i:.2f}" + " +/- " + f"{j:.2f}" for i, j in zip(popt, perr)]
    table_elements = np.split(np.array(table_elements), 3)

    fit_params_widget.data[0].cells.values = \
        [table_elements[2], table_elements[0], table_elements[1]]

display(widget.HBox([residual_widget, hist_widget]), fit_params_widget, minuit_res)

    'data': [{'boxmean': 'sd',
              'boxpoints': False,
              '…

FigureWidget({
    'data': [{'cells': {'align': 'left',
                        'fill': {'color': 'lavender'},…

------------------------------------------------------------------
| FCN = 3.173E+06               |    Ncalls=1779 (1784 total)    |
| EDM = 2.98 (Goal: 1E-05)      |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     False     |     True      |   True    |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   False   |   True    | False  |
------------------------------------------------------------------
------------------------------------------------------------------------------------------
|   | Name |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------
| 0 | x0   |  0.541E5  |  0.000E5  |            |            |    0    |         |       |
| 1 | x1   |  0.483E5  |  0.000E5  |            |            |    0    |         |       |
| 2 | x2   |  0.581E5  |  0.000E5  |            |            |    0    |         |       |
| 3 | x3   |  442.04   |   0.26    |            |            |    0    |         |       |
| 4 | x4   |   563.5   |    0.4    |            |            |    0    |         |       |
| 5 | x5   |  0.649E4  |  0.000E4  |            |            |    0    |         |       |
| 6 | x6   |  -30.846  |   0.005   |            |            |         |         |       |
| 7 | x7   |  11.905   |   0.006   |            |            |         |         |       |
| 8 | x8   |  50.448   |   0.009   |            |            |         |         |       |
------------------------------------------------------------------------------------------

## Data Processing Parallelized over modules

In [ ]:
list_modules = [12, 13, 15]


In [ ]:

dark_average = partial(
    DataProcessing,
    path=dark_run_folder,
    pulse_ids="1:250:2",
    train_index=by_index[:],
    rois=by_index[..., :, :],
    operation=partial(np.mean, axis=0))


futures = OrderedDict()

with ProcessPoolExecutor(max_workers=len(list_modules)) as executor:
    ret = executor.map(dark_average, list_modules)

for data in ret:
    print(data.shape)